<a href="https://colab.research.google.com/github/Communist02/BIN-22-1_Mazur_Denis_ML/blob/master/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install nltk

In [42]:
!pip install pymorphy3

In [43]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import math

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')


class Processing:
    def tokenize(self, text: str) -> list[str]:
        return word_tokenize(text)

    def lematize(self, tokens: list[str]) -> list[str]:
        morph3 = MorphAnalyzer()
        lemmatized_words = [morph3.parse(
            word)[0].normal_form for word in tokens]
        return lemmatized_words

    def stemming(self, tokens: list[str]) -> list[str]:
        stemmer = SnowballStemmer("russian")
        lemmatized_words = [stemmer.stem(word) for word in tokens]
        return lemmatized_words

    def vectorize(self, tokens: list[str]) -> list[int]:
        dict_vectors = {}
        result = []
        for word in tokens:
            if word in dict_vectors.keys():
                result.append(dict_vectors[word])
            else:
                dict_vectors[word] = len(dict_vectors)
                result.append(dict_vectors[word])
        return result

    def vectorize_dict(self, tokens: list[str]) -> list[int]:
        dict_vectors = {}
        result = []
        for word in tokens:
            if word not in dict_vectors.keys():
                dict_vectors[word] = len(dict_vectors)
        return dict_vectors

    def delete_stop_words(self, tokens: list[str]) -> list[int]:
        stop_words = set(stopwords.words('russian')).union(['.', ',', ':', '?', '!'])
        return [word for word in tokens if word not in stop_words]

    def bag_of_words(self, tokens: list[str]) -> dict[str]:
        dict_words = {}
        for word in tokens:
            dict_words[word] = dict_words.setdefault(word, 0) + 1
        return dict_words

    def tf(self, tokens: list[str]) -> dict[str]:
        dict_words = self.bag_of_words(tokens)
        for word in dict_words:
            dict_words[word] /= len(tokens)
        return dict_words

    def idf(self, texts: list[list[str]]) -> dict[str]:
        dict_words = {}
        big_text = []
        for text in texts:
            big_text += list(set(text))
        for word in set(big_text):
            dict_words[word] = math.log(len(texts) / big_text.count(word))
        return dict_words

    def tf_idf(self, texts: list[list[str]], indexText: int) -> dict[str]:
        tf = self.tf(texts[indexText])
        idf = self.idf(texts)
        dict_words = {}
        for word in tf:
            dict_words[word] = tf[word] * idf[word]
        return dict_words


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
import numpy as np

class SimpleTransformer:
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        self.embed_dim = embed_dim

        # Случайные эмбеддинги для токенов
        self.embeddings = np.random.randn(vocab_size, embed_dim)

        # Один "слой внимания" (упрощённо)
        self.W_q = np.random.randn(embed_dim, hidden_dim)
        self.W_k = np.random.randn(embed_dim, hidden_dim)
        self.W_v = np.random.randn(embed_dim, hidden_dim)

        # Выходной слой
        self.W_out = np.random.randn(hidden_dim, vocab_size)

    def softmax(self, x):
        exp = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp / np.sum(exp, axis=-1, keepdims=True)

    def forward(self, input_indices):
        # Получаем эмбеддинги
        x = self.embeddings[input_indices]  # shape: (seq_len, embed_dim)

        # Внимание (один шаг, без маски)
        Q = x @ self.W_q
        K = x @ self.W_k
        V = x @ self.W_v

        attn_scores = Q @ K.T / np.sqrt(self.embed_dim)
        attn_weights = self.softmax(attn_scores)

        attended = attn_weights @ V  # shape: (seq_len, hidden_dim)

        # Среднее по токенам (по сути — пулинг)
        context_vector = np.mean(attended, axis=0)

        # Финальный предикт
        logits = context_vector @ self.W_out  # shape: (vocab_size,)
        probs = self.softmax(logits)
        return probs

    def predict(self, input_indices):
        probs = self.forward(input_indices)
        return np.argmax(probs)


In [47]:
pc = Processing()
text = 'Звезды отражались в её глазах. Раньше, еще месяц назад их не было видно в черте города. Свет фонарей и смог заглушали их слабое мерцание. Все изменилось. И одним из немногих плюсов сложившейся ситуации было мерцание звезд в её глазах, и воздух, кажется стал чище. У всех у нас когда то была работа, и был дом. У некоторых были дети. У Лены была дочка. Она работала барменшой, а по вечерам подрабатывала в "клубе знакомств". Попросту говоря - была проституткой. Теперь ей уже не приходится ездить по незнакомым клиентам, каждый раз перед дверью квартиры креститься, и молиться, что бы все прошло как надо. Это тоже плюс. Но теперь у неё нет дочки. Она потерялась в первые дни, как только все это начиналось. Лена была "на вызове", когда исчезло электричество. Никто еще не знал, что это серьезно. Мобильная связь не работала, город погрузился во тьму за окнами однакомнатной квартиры, в которой возбужденный мужчина кончал в презерватив, а Лена считала секунды до очередного вызова. Она не могла как обычно принять душ, и вызвать такси, и после осознания этого, просто начала одеваться. Белье по привычке было сложено одной кучкой рядом с кроватью. Мужчина, имя которого она не захотела запоминать сказал ей спасибо и открыл дверь, что то проворчав напоследок на "долбанных электриков"... Лене было очень приятно выйти на свежий воздух, после пропахшей перегаром комнатушки. Она шла по темным улицам города, шла на "базу" пешком, и эта непроглядная тьма вокруг для неё сейчас была отражением внутреннего состояния, и поэтому она наслаждалась этой прогулкой. Она еще не знала, что электричество и водоснабжение уже не восстановят. Она не могла подумать, что через три часа её пятилетняя дочка, испугавшись темноты и одиночества, выйдет из квартиры, и пропадет навсегда. Она еще не знала, что её поиски будут бесполезны и опасны... Она просто шла по улице.'

orders = [
    'Звезды отражались',
    'Воздух стал',
    'Она раньше шла',
    'Дочка испугалась',
    'Город погрузился',
    'Она не могла месяц',
    'Электричество исчезло',
    'Поиск видно'
]

result = [
    'глазах',
    'чище',
    'темным',
    'одиночества',
    'тьму',
    'связь',
    'связь',
    'опасен'
]

tokens = pc.delete_stop_words(pc.lematize(pc.tokenize(text)))

# Векторизация словаря
vocab = list(set(tokens))
word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}

# Пример данных
X = []
y = []

# Задача - предсказать последнее слово
for phrase, label in zip(orders, result):
    tokens = pc.delete_stop_words(pc.lematize(pc.tokenize(phrase)))
    if len(tokens) < 2: continue
    input_indices = [word_to_index.get(w, 0) for w in tokens if w in word_to_index]
    label_index = word_to_index.get(label, 0)
    X.append(input_indices)
    y.append(label_index)

# Обучение
model = SimpleTransformer(vocab_size=len(vocab), embed_dim=16, hidden_dim=32)

for epoch in range(10000):
    loss = 0
    for i in range(len(X)):
        probs = model.forward(X[i])
        target = y[i]
        loss += -np.log(probs[target] + 1e-9)  # Кросс-энтропия
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss / len(X)}")

# Предсказания
print("Predictions:")
for phrase in orders:
    tokens = pc.delete_stop_words(pc.lematize(pc.tokenize(phrase)))
    input_indices = [word_to_index.get(w, 0) for w in tokens if w in word_to_index]
    pred_index = model.predict(input_indices)
    print(f"{phrase} -> {index_to_word[pred_index]}")


Epoch 0, Loss: 19.26107186649898
Epoch 100, Loss: 19.26107186649898
Epoch 200, Loss: 19.26107186649898
Epoch 300, Loss: 19.26107186649898
Epoch 400, Loss: 19.26107186649898
Epoch 500, Loss: 19.26107186649898
Epoch 600, Loss: 19.26107186649898
Epoch 700, Loss: 19.26107186649898
Epoch 800, Loss: 19.26107186649898
Epoch 900, Loss: 19.26107186649898
Epoch 1000, Loss: 19.26107186649898
Epoch 1100, Loss: 19.26107186649898
Epoch 1200, Loss: 19.26107186649898
Epoch 1300, Loss: 19.26107186649898
Epoch 1400, Loss: 19.26107186649898
Epoch 1500, Loss: 19.26107186649898
Epoch 1600, Loss: 19.26107186649898
Epoch 1700, Loss: 19.26107186649898
Epoch 1800, Loss: 19.26107186649898
Epoch 1900, Loss: 19.26107186649898
Epoch 2000, Loss: 19.26107186649898
Epoch 2100, Loss: 19.26107186649898
Epoch 2200, Loss: 19.26107186649898
Epoch 2300, Loss: 19.26107186649898
Epoch 2400, Loss: 19.26107186649898
Epoch 2500, Loss: 19.26107186649898
Epoch 2600, Loss: 19.26107186649898
Epoch 2700, Loss: 19.26107186649898
Epoc